# Using the MANN Package to convert and prune an existing TensorFlow model

In this notebook, we utilize the MANN package on an existing TensorFlow model to convert existing layers to MANN layers and then prune the model.

In [1]:
# Load the MANN package and TensorFlow
import tensorflow as tf
import beyondml.tflow as mann

In [2]:
# Load the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train/255
x_test = x_test/255

# Load the model to be used
vgg16 = tf.keras.applications.VGG16(
    include_top = False,             # Don't include the top layers
    weights = 'imagenet',            # Load the imagenet weights
    input_shape = x_train.shape[1:]  # Input shape is the shape of the images
)

2022-03-03 10:45:34.442895: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-03 10:45:34.442998: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



## Create the model to be trained

In the following cell, we create the model using the existing VGG model fed into fully-connected layers.

In [3]:
# Build the model using VGG16 and a few layers on top of it
model = tf.keras.models.Sequential()
model.add(vgg16)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation = 'relu'))
model.add(tf.keras.layers.Dense(512, activation = 'relu'))
model.add(tf.keras.layers.Dense(512, activation = 'relu'))
model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

# Compile the model
model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

# Present model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 15,507,786
Trainable params: 15,507,786
Non-trainable params: 0
____________________________________________

## Convert the model and perform initial pruning

In the following cell, we convert the model and perform initial pruning of the model to 40%.

In [4]:
# Use the add_layer_masks function to add masking layers to the model
converted_model = mann.utils.add_layer_masks(model)

# Compile the model
converted_model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

# Mask the model using magnitude as the metric
converted_model = mann.utils.mask_model(
    converted_model,
    40,
    method = 'magnitude'
)

# Recompile the model for the weights to take effect
converted_model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

# Present the model summary
converted_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         29429376  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
masked_dense (MaskedDense)   (None, 512)               525312    
_________________________________________________________________
masked_dense_1 (MaskedDense) (None, 512)               525312    
_________________________________________________________________
masked_dense_2 (MaskedDense) (None, 512)               525312    
_________________________________________________________________
masked_dense_3 (MaskedDense) (None, 10)                10260     
Total params: 31,015,572
Trainable params: 15,507,786
Non-trainable params: 15,507,786
___________________________________

## Train and further prune the model

In this cell, we create the ActiveSparsification callback and train the model using that callback to prune the model as the model improves in performance.

In [5]:
# Create the sparsification callback object
callback = mann.utils.ActiveSparsification(
    performance_cutoff = 0.75,    # The accuracy score the model needs to achieve
    starting_sparsification = 40, # Starting sparsification
    sparsification_rate = 5       # Sparsification increase every time the model achieves performance cutoff
)

# Fit the model
model.fit(
    x_train,
    y_train,
    epochs = 1000,
    callbacks = [callback],
    validation_split = 0.2,
    batch_size = 256
)

2022-03-03 10:45:35.917442: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-03-03 10:45:35.918470: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/1000


2022-03-03 10:45:36.127297: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


157/157 [==============================] - ETA: 0s - loss: 1.9748 - accuracy: 0.2022

2022-03-03 10:46:28.293310: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


157/157 [==============================] - 56s 354ms/step - loss: 1.9748 - accuracy: 0.2022 - val_loss: 1.7803 - val_accuracy: 0.2515
Performance measure set to val_accuracy
Model performance has not reached pruning threshold for 1 epoch(s)
Epoch 2/1000
157/157 [==============================] - 55s 349ms/step - loss: 1.5569 - accuracy: 0.3574 - val_loss: 1.4051 - val_accuracy: 0.4592
Model performance has not reached pruning threshold for 2 epoch(s)
Epoch 3/1000
157/157 [==============================] - 55s 348ms/step - loss: 1.1886 - accuracy: 0.5540 - val_loss: 1.0138 - val_accuracy: 0.6401
Model performance has not reached pruning threshold for 3 epoch(s)
Epoch 4/1000
157/157 [==============================] - 55s 348ms/step - loss: 0.8670 - accuracy: 0.6950 - val_loss: 0.8634 - val_accuracy: 0.6979
Model performance has not reached pruning threshold for 4 epoch(s)
Epoch 5/1000
157/157 [==============================] - 55s 350ms/step - loss: 0.6957 - accuracy: 0.7677 - val_loss: 

## Convert the model back to remove masking layers

In the following cell, we remove the layer masks created for training, while completely preserving performance.

In [6]:
# Convert the model back
model = mann.utils.remove_layer_masks(model)

# Present the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 15,507,786
Trainable params: 0
Non-trainable params: 15,507,786
____________________________________________

## Report accuracy and save model

In [7]:
# Get the predictions on test data
preds = model.predict(x_test).argmax(axis = 1)

# Print the accuracy
print(f'Model Accuracy: {(preds.flatten() == y_test.flatten()).sum().astype(int)/y_test.flatten().shape[0]}')

# Save the model
model.save('cifar_vgg16.h5')

2022-03-03 11:04:54.104795: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Model Accuracy: 0.8001
